In [1]:
! pip install spacy sentence-transformers PyMuPDF gradio scikit-learn --quiet
!python -m spacy download en_core_web_sm --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 125.4 MB/s eta 0:00:00
✔ Download and installation successful
You

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re
import spacy
import fitz  # PyMuPDF


In [3]:
# Load spaCy NER model
nlp = spacy.load("en_core_web_sm")

# Load Sentence-BERT (E5 is more advanced than original SBERT)
model = SentenceTransformer("intfloat/e5-large-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [4]:



def clean_text(text, lowercase=False):
    # Remove newlines and tabs
    text = re.sub(r"[\n\t]+", " ", text)

    # Remove multiple spaces
    text = re.sub(r"\s{2,}", " ", text)

    # Remove common footer/header like "Page 1 of 2"
    text = re.sub(r"page\s+\d+\s+of\s+\d+", "", text, flags=re.I)

    return text.strip()

In [5]:


def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return clean_text(text)

In [6]:
def extract_info(text, user_name=""):
    # Email and phone regex
    email = re.findall(r"\S+@\S+", text)
    phone = re.findall(r"\+?\d[\d\- ]{9,}\d", text)

    return {
        "name": user_name if user_name else None,
        "email": email[0] if email else None,
        "phone": phone[0] if phone else None,
    }


In [7]:
def split_sections(text):
    pattern = r"(?i)\b(Skills|Experience|Education|Projects|Certifications|Summary|Objective)\b"
    parts = re.split(pattern, text)
    structured = {}
    current_key = "Other"
    structured[current_key] = ""
    for part in parts:
        part = part.strip()
        if re.fullmatch(pattern, part, flags=re.I):
            current_key = part.title()
            structured[current_key] = ""
        else:
            structured[current_key] += part + " "
    return structured

In [8]:
def match_jobs(resume_text, job_descriptions):
    resume_embedding = model.encode(f"query: {resume_text}", convert_to_tensor=True)
    job_embeddings = model.encode([f"passage: {desc}" for desc in job_descriptions], convert_to_tensor=True)

    scores = cosine_similarity([resume_embedding.cpu().numpy()], job_embeddings.cpu().numpy())[0]
    sorted_idx = scores.argsort()[::-1]

    result = ""
    for i in sorted_idx:
        result += f"🔹 **Score**: {scores[i]:.4f}\n**Job**: {job_descriptions[i]}\n\n"

    return result

In [9]:
def resume_matcher(pdf_file, user_name, job_input):
    if pdf_file is None or not job_input.strip():
        return "❌ Please upload a resume and enter at least one job description."

    # Extract text from PDF
    raw_text = extract_text_from_pdf(pdf_file)

    # Extract other info (email + phone only)
    info = extract_info(raw_text, user_name)

    # Focus on Skills + Experience
    sections = split_sections(raw_text)
    resume_text = sections.get("Skills", "") + " " + sections.get("Experience", "")

    # Clean job descriptions
    job_descriptions = [j.strip() for j in job_input.strip().split("\n") if j.strip()]
    if not job_descriptions:
        return "❌ Please provide at least one job description."

    # Perform matching
    match_result = match_jobs(resume_text, job_descriptions)

    profile = f"""### 👤 Extracted Info:
- **Name**: {info['name']}
- **Email**: {info['email']}
- **Phone**: {info['phone']}

### ✅ Top Job Matches:
{match_result}
"""
    return profile


In [11]:
import gradio as gr


In [12]:
gr.Interface(
    fn=resume_matcher,
    inputs=[
        gr.File(label="Upload Resume (PDF)"),
        gr.Textbox(label="Enter Your Name"),
        gr.Textbox(lines=8, label="Paste Job Descriptions (one per line)")
    ],
    outputs=gr.Markdown(label="Match Results"),
    title="📄 AI-Powered Resume Matcher",
    description="Upload a resume and paste job descriptions. The app matches them using Sentence-BERT (E5) and cosine similarity."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7084a4aefe13087149.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
